In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

df = pd.read_csv('data/kc_house_data.csv')

In [2]:
# make subsets of the columns
numeric_continuous = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_garage', 'sqft_patio']
numeric_discrete = ['bedrooms', 'bathrooms', 'floors', 'yr_built', 'yr_renovated']
categorical = ['waterfront', 'greenbelt', 'nuisance', 'view', 'condition', 'grade', 'heat_source', 'sewer_system']

In [3]:
df_numeric_continuous = df[numeric_continuous].copy()
for column in numeric_continuous:
    df_numeric_continuous[column] = df_numeric_continuous[column] - df_numeric_continuous[column].mean()

df_numeric_discrete = df[numeric_discrete].copy()

df_categorical = df[categorical].copy()
df_categorical = pd.get_dummies(df_categorical, columns = categorical)
df_categorical.drop(columns = ['waterfront_NO', 'greenbelt_NO', 'nuisance_NO', 'view_NONE',
                               'condition_Average', 'grade_7 Average', 'heat_source_Other'], axis = 1, inplace = True)

In [7]:
y = df['price']
X_baseline = pd.concat([df_numeric_continuous, df_numeric_discrete, df_categorical], axis = 1)

baseline_model = sm.OLS(y, sm.add_constant(X_baseline))
baseline_results = baseline_model.fit()

print(baseline_results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.528
Model:                            OLS   Adj. R-squared:                  0.527
Method:                 Least Squares   F-statistic:                     766.0
Date:                Tue, 21 Feb 2023   Prob (F-statistic):               0.00
Time:                        14:00:44   Log-Likelihood:            -4.4477e+05
No. Observations:               30155   AIC:                         8.896e+05
Df Residuals:                   30110   BIC:                         8.900e+05
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     